# Analyse de données et modèle de prédiction

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

### Initialize Spark session

In [ ]:
spark = SparkSession.builder \
    .appName("Vehicle Category Prediction") \
    .getOrCreate()

In [ ]:
def print_df(df):
    print("Size", df.count())
    df.printSchema()
    df.show(5)

def load_data_in_csv_file(file_path, delimit = ",", head=True):
    df = spark.read.options(delimiter=",", header=True, inferSchema=True).csv(file_path)

    # Display schema and first five rows of the DataFrame
    print("Schema and first rows in", file_path)
    print_df(df)
    
    return df

### Load the dataset

In [ ]:
# File path in Hadoop FS
hdfs_path = "hdfs:///tpa_groupe_14/data/fusion/fusion.csv"

# Load CSV data into a DataFrame
vehicles_df = load_data_in_csv_file(hdfs_path)

### Data formatting

In [ ]:
column_names = vehicles_df.schema.names

for column_name in column_names:
    vehicles_df = vehicles_df.withColumnRenamed(column_name, column_name.split(".")[-1])

print_df(vehicles_df)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer

### Optional

In [ ]:
# Optional: Sub-sample the DataFrame
vehicles_df = vehicles_df.sample(withReplacement=False, fraction=0.3, seed=42)

In [ ]:
train_df.show(5)

In [ ]:
# Prepare features
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_index").fit(vehicles_df)
    for column in ["sexe", "situationfamiliale", "categorie"]
]

assembler = VectorAssembler(
    inputCols=[
        "age", "taux", "nbenfantsacharge", "deuxiemevoiture",
        "sexe_index", "situationfamiliale_index"
    ],
    outputCol="features"
)

# Split the DataFrame into training and test sets (80% training, 20% test)
train_df, test_df = vehicles_df.randomSplit([0.8, 0.2], seed=42)

pipeline = Pipeline(stages=indexers + [assembler])
pipeline_model = pipeline.fit(train_df)

train_df = pipeline_model.transform(train_df)
test_df = pipeline_model.transform(test_df)

In [ ]:
label_predictions = test_df.select("categorie", "categorie_index").distinct().rdd.collect()
categorie_dict = {row['categorie_index']: row['categorie'] for row in label_predictions}

print(categorie_dict)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import MulticlassMetrics

# La précision & Matrice de confusion

In [ ]:
# Evaluation function
def evaluate_classifier(classifier, param_grid, train_df, test_df, title):
    evaluator = MulticlassClassificationEvaluator(labelCol="categorie_index", predictionCol="prediction", metricName="accuracy")
    
    # Create the cross-validator
    cross_validator = CrossValidator(
        estimator=classifier,
        estimatorParamMaps=param_grid,
        evaluator=evaluator,
        numFolds=3, 
        seed=42
    )
    
    # Train the model with the best hyperparameters
    cv_model = cross_validator.fit(train_df)
    
    # Make predictions on the test data
    predictions = cv_model.transform(test_df)
    
    # Evaluate the model
    accuracy = evaluator.evaluate(predictions)
    
    # Calcul de la matrice de confusion
    predictions_and_labels = predictions.select("prediction", "categorie_index")
    prediction_rdd = predictions_and_labels.rdd.map(tuple)
    
    metrics = MulticlassMetrics(prediction_rdd)
    confusion_matrix = metrics.confusionMatrix().toArray()

    # Calcul des taux de succès par classe
    labels = predictions.select("categorie_index").distinct().rdd.flatMap(lambda x: x).collect()
    
    per_class_metrics = {}
    for label in labels:
        per_class_metrics[label] = {
            "precision": metrics.precision(label),
            "recall": metrics.recall(label),
            "f1_score": metrics.fMeasure(label)
        }
    
    # Calcul des métriques globales pondérées
    weighted_metrics = {
        "precision": metrics.weightedPrecision,
        "recall": metrics.weightedRecall,
        "f1_score": metrics.weightedFMeasure(),
        "accuracy": accuracy
    }

    print(title + " Accuracy = {:.2f}".format(accuracy))
    print("\nWeighted Metrics:")
    print(f"  Weighted Precision: {weighted_metrics['precision']:.2f}")
    print(f"  Weighted Recall: {weighted_metrics['recall']:.2f}")
    print(f"  Weighted F1 Score: {weighted_metrics['f1_score']:.2f}")
    print(f"  Accuracy: {weighted_metrics['accuracy']:.2f}")
    
    return {
        "classifier": title,
        "cv_model": cv_model,
        "accuracy": accuracy,
        "confusion_matrix": confusion_matrix,
        "per_class_metrics": per_class_metrics,
        "weighted_metrics": weighted_metrics
    }

In [ ]:
from pyspark.ml.classification import (
    DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, 
    LinearSVC, LogisticRegression, MultilayerPerceptronClassifier, NaiveBayes, OneVsRest
)

In [ ]:
def calculate_composite_score(metrics, weights):
    score = 0
    
    for metric, value in metrics.items():
        score += weights.get(metric, 0) * value
        
    return score

In [ ]:
def compare_classifiers(classifiers, param_grids, train_df, test_df, titles):
    results = []
    
    for classifier, param_grid, title in zip(classifiers, param_grids, titles):
        print(f"\nEvaluating {title}...\n")
        result = evaluate_classifier(classifier, param_grid, train_df, test_df, title)
        weighted_metrics = result['weighted_metrics']
        composite_score = calculate_composite_score(weighted_metrics, weighted_metrics)
        results.append({ "composite_score": composite_score, "cv_model": result["cv_model"], "classifier": result["classifier"] })
    
    # Rank the models based on composite scores
    ranked_results = sorted(results, key=lambda item: item["composite_score"], reverse=True)

    print("Model with highest score:", ranked_results[0]["classifier"], "[", ranked_results[0]["composite_score"], "]")
    
    return ranked_results[0]["cv_model"]

## Entrainement et comparaison des modèles

In [ ]:
rf = RandomForestClassifier(labelCol="categorie_index", featuresCol="features")

# Define the hyperparameter grid
rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

dt = DecisionTreeClassifier(labelCol="categorie_index", featuresCol="features")

dt_param_grid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10]) \
    .addGrid(dt.impurity, ["gini", "entropy"]) \
    .build()

gbt = GBTClassifier(labelCol="categorie_index", featuresCol="features")
gbt_ovr = OneVsRest(classifier=gbt, labelCol="categorie_index", featuresCol="features")

gbt_param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [10, 20]) \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .build()

svm = LinearSVC(labelCol="categorie_index", featuresCol="features")
svm_ovr = OneVsRest(classifier=svm, labelCol="categorie_index", featuresCol="features")

svm_param_grid = ParamGridBuilder() \
    .addGrid(svm.maxIter, [10, 20]) \
    .addGrid(svm.regParam, [0.01, 0.1]) \
    .build()

lr = LogisticRegression(labelCol="categorie_index", featuresCol="features")

lr_param_grid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [10, 20]) \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

num_classes = train_df.select("categorie_index").distinct().count()

mlp = MultilayerPerceptronClassifier(labelCol="categorie_index", featuresCol="features", layers=[6, 5, 4, num_classes])

mlp_param_grid = ParamGridBuilder() \
    .addGrid(mlp.maxIter, [50, 100]) \
    .build()

nb = NaiveBayes(labelCol="categorie_index", featuresCol="features")

nb_param_grid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.5, 1.0, 1.5]) \
    .build()

classifiers = [
    rf,
    dt, 
    gbt_ovr,
    svm_ovr,
    lr,
    mlp,
    nb
]

param_grids = [
    rf_param_grid,
    dt_param_grid, 
    gbt_param_grid,
    svm_param_grid,
    lr_param_grid,
    mlp_param_grid,
    nb_param_grid
]

titles = [
    "RandomForestClassifier",
    "DecisionTreeClassifier", 
    "GBTClassifier",
    "LinearSVC",
    "LogisticRegression",
    "MultilayerPerceptronClassifier",
    "NaiveBayes"
]

# Assuming train_df and test_df are already defined
model = compare_classifiers(classifiers, param_grids, train_df, test_df, titles)

# Prediction

- Charger Marketing

In [ ]:
from pyhive import hive
import pandas as pd

In [ ]:
hive_host = 'localhost'
hive_port = 10000
hive_username = ' ' 
hive_password = ' '

In [ ]:
#conn = None
try:
    # Établir une connexion avec authentification LDAP
    conn = hive.Connection(
        host=hive_host,
        port=hive_port,
        username=hive_username,
        password=hive_password,
        auth='LDAP'  
    )
    print("Connecté à Hive avec succès")
except Exception as e:
    print(f"Erreur lors de la connexion à Hive: {e}")

In [ ]:
# Créer un curseur
cursor=conn.cursor()

In [ ]:
def findAll(table) :
    # Exécuter une requête pour récupérer les données de la table "catalogue"
    query = "SELECT * FROM " + table
    cursor.execute(query)

    # Charger les résultats dans un DataFrame Pandas
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description] 
    df = spark.createDataFrame(data, schema=columns)
    # df = spark.createDataFrame(pd_df)
    
    return df

In [ ]:
marketing_df = findAll("marketing_view")

Prétraiter les Données Marketing

In [ ]:
# Renommer les colonnes pour éviter les conflits
for column_name in marketing_df.columns:
    marketing_df = marketing_df.withColumnRenamed(column_name, column_name.split(".")[-1])

In [ ]:
marketing_df.show(5)

Appliquer le Modèle de Prédiction

In [ ]:
marketing_df = pipeline_model.transform(marketing_df)

In [ ]:
marketing_df.show(5)

In [ ]:
predictions = model.transform(marketing_df)

In [ ]:
# Sélectionner les colonnes pertinentes pour l'affichage
for key, value in categorie_dict.items():
    predictions = predictions.withColumn("prediction", when(col('prediction') == key, value).otherwise(col('prediction')))

results = predictions.select("id", "prediction")

results.show()

Exporter les Résultats

In [ ]:
# Chemin de sortie pour les résultats
results_hdfs_path = "hdfs:///tpa_groupe_14/results"

# Sauvegarder les résultats dans HDFS
results.write.csv(results_hdfs_path, header=True)

In [ ]:
!hadoop fs -ls /tpa_groupe_14/results